# CONNECT THE BOARD

In [35]:
%serialconnect to --port="COM5" --baud=115200

serial exception on close WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))
Connecting to --port=COM5 --baud=115200 
Ready.


In [36]:
f = open("main.py", "r")
print(f.read())

import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 200,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT

In [ ]:
%sendtofile lib/collector.py --source lib/collector.py
%sendtofile lib/peripherals.py --source lib/peripherals.py
%sendtofile lib/signal.py --source lib/signal.py

In [14]:
%sendtofile main.py --source main.py

Sent 74 lines (1789 bytes) to main.py.


# COLLECTOR METHOD

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

from lib.requests import MicroWebCli as requests
import ujson as json

from lib.utils import connect_wifi, load_env_vars

from ulab import numpy as np
import utime as time
from lib.collector import Collector 
from lib.runner import Runner 

In [ ]:
collector = Collector(buffer_size=256, preprocess=True, ds=True)

SPI_params= {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,
}

collector.setup(spi_params=SPI_params)

# ssid = 'TP-Link_AP_4C04'
# password = '63525465'
# connect_wifi(ssid, password)
time.sleep(4)

collector.run()

time.sleep(10)

while True:
    time.sleep(1)
    bt = time.time()
    data = collector.read_output_buffer()
    et = time.time()
    print(et-bt,data)
#     toSend = {"raw_data":data}
#     print(toSend)
#     requests.JSONRequest("http://192.168.0.13:5001/collect", toSend)
#     # requests.JSONRequest("http://172.20.10.2:5001/collect", toSend)
#     del data
#     del toSend
#     # requests.GETRequest("http://172.20.10.2:5001/save")
#     requests.GETRequest("http://192.168.0.13:5001/save")

In [ ]:
collector.stop()

In [ ]:
runner = Runner('CCA', buffer_size=256)
runner.setup()

# ssid = 'VM0857951'
# password = 'cchk7mrYgrzq'
# connect_wifi(ssid, password)

time.sleep(4)
runner.run()
while True:
    time.sleep(1)
    data = runner.read_output_buffer()
    print(data)

In [ ]:
runner.stop()

# DIRECTLY READ ADC, SKIP EVERYTHING ELSE

In [ ]:
%serialconnect to --port="COM5" --baud=115200

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    **temp_spi_params
)

data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

# print(adc_sample)
# print("20000 ADC readings done after %u us." %(t2-t1))
# print("Mean time for each ADC reading = %15.13f us" % ((t2-t1)/20000.0))
# print("ADC reading = %15.13f ksamples/s" % (1000/((t2-t1)/20000.0)))

In [ ]:
while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

In [12]:
requests.GETRequest("http://192.168.0.37:5001/save")

[leftinbuffer] ['659, 1840, 1936, 1776, 1599, 1638, 1817, 1914, 1791, 1610, 1595, 1759, 1893, 1792, 1607, 1569, 1727, 1873, 1808, 1616, 1552, 1695, 1861, 1830, 1646, 1552, 1677, 1852, 1849, 1639, 1552, 1682, 1843, 1872, 1699, 1569, 1648, 1839, 1901, 1737, 1593, 1649, 1840, 1929, 1778, 1618, 1651, 1831, 1939, 1808, 1637, 1639, 1808, 1935, 1820, 1600, 1607, 1792, 1909, 1823, 1627, 1571, 1719, 1873, 1821, 1633, 1552, 1687, 1857, 1840, 1657, 1555, 1671, 1847, 1855, 1674, 1547, 1641, 1826, 1870, 1697, 1557, 1630, 1845, 1878, 1727, 1577, 1617, 1802, 1904, 1765, 1605, 1619, 1801, 1923, 1803, 1632, 1627, 1801, 1937, 1843, 1659, 1622]}']
[leftinbuffer] ['24768']
[leftinbuffer] ["{'raw_data': [1871, 1795, 1616, 1598, 1744, 1901, 1745, 1649, 1598, 1744, 1914, 1885, 1699, 1611, 1741, 1916, 1911, 1727, 1611, 1715, 1899, 1920, 1741, 1601, 1680, 1859, 1909, 1718, 1577, 1643, 1824, 1899, 1741, 1580, 1599, 1774, 1879, 1744, 1574, 1580, 1749, 1877, 1771, 1589, 1565, 1724, 1872, 1795, 1615, 1563, 1717, 1

In [7]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [5]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

network config: ('192.168.0.84', '255.255.255.0', '192.168.0.1', '194.168.4.100')
71392
{'raw_data': [1787, 1665, 1623, 1739, 1861, 1831, 1690, 1638, 1753, 1867, 1851, 1708, 1633, 1718, 1853, 1858, 1717, 1628, 1695, 1827, 1856, 1717, 1616, 1664, 1799, 1850, 1718, 1601, 1635, 1776, 1842, 1732, 1603, 1621, 1763, 1840, 1749, 1614, 1613, 1748, 1851, 1781, 1644, 1616, 1742, 1850, 1797, 1655, 1613, 1728, 1854, 1823, 1680, 1618, 1720, 1850, 1842, 1702, 1626, 1712, 1865, 1862, 1716, 1629, 1697, 1836, 1870, 1729, 1621, 1671, 1808, 1861, 1732, 1613, 1643, 1779, 1853, 1744, 1611, 1621, 1756, 1840, 1749, 1615, 1609, 1741, 1836, 1775, 1634, 1607, 1733, 1846, 1799, 1655, 1609, 1725, 1851, 1826, 1682, 1616, 1715, 1854, 1849, 1705, 1622, 1708, 1846, 1866, 1727, 1634, 1701, 1841, 1876, 1733, 1633, 1680, 1817, 1872, 1751, 1625, 1657, 1793, 1866, 1761, 1622, 1636, 1770, 1856, 1769, 1630, 1619, 1748, 1850, 1778, 1637, 1606, 1726, 1840, 1790, 1638, 1600, 1709, 1831, 1806, 1661, 1595, 1696, 1828, 1824, 1681

In [23]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 50,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > pot_size:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

connecting to network...
network config: ('192.168.0.84', '255.255.255.0', '192.168.0.1', '194.168.4.100')
62976
{'raw_data': [1729, 1727, 1719, 1709, 1699, 1708, 1723, 1743, 1761, 1771, 1763, 1747, 1727, 1712, 1701, 1707, 1713, 1741, 1757, 1767, 1769, 1751, 1731, 1713, 1702, 1703, 1712, 1746, 1751, 1765, 1767, 1761, 1743, 1723, 1711, 1708, 1714, 1735, 1755, 1771, 1776, 1766, 1749, 1727, 1712, 1709, 1709, 1718, 1742, 1751, 1764, 1755, 1743, 1715, 1709, 1701, 1706, 1717, 1739, 1754, 1762, 1759, 1749, 1729, 1712, 1699, 1707, 1718, 1737, 1754, 1765, 1761, 1739, 1728, 1712, 1707, 1708, 1717, 1735, 1755, 1769, 1771, 1760, 1742, 1718, 1707, 1702, 1710, 1728, 1748, 1765, 1775, 1763, 1744, 1725, 1711, 1702, 1710, 1726, 1762, 1761, 1762, 1759, 1745, 1725, 1697, 1698, 1702, 1714, 1739, 1755, 1767, 1763, 1751, 1729, 1711, 1697, 1698, 1710, 1727, 1745, 1761, 1763, 1751, 1734, 1714, 1703, 1699, 1712, 1727, 1745, 1772, 1774, 1765, 1744, 1726, 1711, 1707, 1712, 1729, 1751, 1767, 1776, 1770, 1753, 173

In [42]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

# TESTING GAIN

In [49]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [28]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)
cs.on() #active low
print("Set GPIO 5 LOW")
spi.write(data)
print("Sent gain")
cs.off() #active high
print("Set GPIO 5 HIGH")
while True:
    time.sleep(1)
    print(adc.read())

Set GPIO 5 LOW
Sent gain
Set GPIO 5 HIGH
1693
1684
1730
1349
1298
2783
1757
1192
3747
.0
2088
0
1791
1454
1623
1494
1328
1328
2094
.1978
2081
2007
1567
1277
1371
3743
2436
1876
1670
.1585
1665
1690
1727


*** Sending Ctrl-C

775


Traceback (most recent call last):
  File "<stdin>", line 49, in <module>
KeyboardInterrupt: 


In [16]:
# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
data = bytearray(255)
spi.write(data)

In [50]:
%sendtofile main.py --source main.py

Sent 85 lines (2052 bytes) to main.py.


In [25]:
import machine
from machine import Pin
from machine import ADC, SPI
from machine import freq

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

In [16]:
%ls lib

Listing directory 'lib'.
     8196    lib/.DS_Store
        0    lib/__init__.py
      446    lib/config.py
     8871    lib/decoding.py
      855    lib/diagnostics.py
     3503    lib/logging.py
     1254    lib/networking.py
     5697    lib/peripherals.py
     8212    lib/runner.py
      384    lib/scheduling.py
     2067    lib/signal.py
     1573    lib/synthetic.py
     6570    lib/umqtt.py
     1691    lib/utils.py
      976    lib/websockets.py


In [ ]:
%sendtofile lib/computation.py --source lib/computation.py

# BEFORE CHANGES TO GAIN

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 100,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > pot_size:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

# after changes to gain - testing

In [48]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to pull GPIO 5 LOW before writing the gain

cs.on() #active low
spi.write(data)
cs.off() #active high

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

network config: ('192.168.0.84', '255.255.255.0', '192.168.0.1', '194.168.4.100')
82128
{'raw_data': [1726, 1711, 1683, 1669, 1681, 1712, 1751, 1778, 1795, 1757, 1751, 1714, 1686, 1666, 1677, 1700, 1741, 1770, 1792, 1787, 1765, 1728, 1694, 1671, 1673, 1694, 1731, 1763, 1789, 1790, 1772, 1736, 1702, 1676, 1669, 1684, 1717, 1755, 1780, 1790, 1775, 1744, 1707, 1681, 1666, 1680, 1710, 1747, 1775, 1791, 1778, 1755, 1714, 1691, 1668, 1679, 1701, 1739, 1767, 1789, 1782, 1755, 1726, 1699, 1677, 1676, 1693, 1729, 1759, 1782, 1785, 1770, 1739, 1703, 1680, 1674, 1687, 1716, 1754, 1776, 1785, 1775, 1744, 1710, 1686, 1671, 1683, 1712, 1746, 1773, 1789, 1776, 1754, 1719, 1691, 1674, 1679, 1702, 1743, 1771, 1791, 1783, 1765, 1729, 1700, 1678, 1677, 1697, 1729, 1761, 1783, 1785, 1771, 1713, 1704, 1680, 1677, 1690, 1723, 1759, 1779, 1792, 1776, 1747, 1712, 1688, 1671, 1683, 1710, 1747, 1773, 1789, 1781, 1757, 1718, 1694, 1673, 1680, 1702, 1739, 1766, 1787, 1779, 1761, 1728, 1699, 1677, 1678, 1695, 1727

In [45]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.
